In [ ]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()
session = os.getenv('session')

# Replace 'your_session_cookie_here' with your actual session cookie
cookie = {'session': session}

# Replace with the specific day and year of the puzzle
url = 'https://adventofcode.com/2023/day/5/input'

response = requests.get(url, cookies=cookie)

if response.status_code != 200:
    print(f"Error: {response.status_code}")   

In [ ]:
from tqdm import tqdm 
info = 0
seed_to_soil = {}
seed_to_fertilizer = {}
seed_to_water = {}
seed_to_light = {}
seed_to_temperature = {}
seed_to_humidity = {}
seed_to_location = {}

def mapping_info(input_list, dict_1, dict_2):
    if len(input_list) == 3:
        input_list = [int(i) for i in input_list]
        for seed in dict_1.keys():
            if input_list[1] <= dict_1[seed] and dict_1[seed] < input_list[1] + input_list[2]:
                # print(f"{seed} in range {input_list[1]} to {input_list[0]} and diff to {(seed - input_list[1])}")
                dict_2[seed] = (dict_1[seed] - input_list[1]) + input_list[0]
    if len(input_list) == 0:
            for seed in dict_1.keys():
                dict_2[seed] = dict_2.get(seed, dict_1[seed])
    return dict_2

input_text = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""
input_text=response.text
for sentence in tqdm(input_text.split("\n")):
    if sentence[:6]=="seeds:":
        seeds = [int(i) for i in sentence.split(": ")[1].split()]
        # new_seeds = []
        # for i in tqdm(range(len(seeds)//2)):
        #     # print(list(range(seeds[i*2], seeds[i*2] + seeds[i*2+1])))
        #     new_seeds.extend(list(range(seeds[i*2], seeds[i*2] + seeds[i*2+1])))
        # seeds = list(set(new_seeds))
        # print(new_seeds)
    if sentence == "seed-to-soil map:":
        info = 1
    if sentence == "soil-to-fertilizer map:":
        info = 2
    if sentence == "fertilizer-to-water map:":
        info = 3
    if sentence == "water-to-light map:":
        info = 4
    if sentence == "light-to-temperature map:":
        info = 5
    if sentence == "temperature-to-humidity map:":
        info = 6
    if sentence == "humidity-to-location map:":
        info = 7

    if info == 1:
        input_list = sentence.split()
        if len(input_list) == 3:
            input_list = [int(i) for i in input_list]
            for seed in seeds:
                if input_list[1] <= seed and seed < input_list[1] + input_list[2]:
                    seed_to_soil[seed] = (seed - input_list[1]) + input_list[0]
        if len(input_list) == 0:
            for seed in seeds:
                seed_to_soil[seed] = seed_to_soil.get(seed, seed)
    
    if info == 2:
        input_list = sentence.split()
        seed_to_fertilizer = mapping_info(input_list, seed_to_soil, seed_to_fertilizer)

    if info == 3:
        input_list = sentence.split()
        seed_to_water = mapping_info(input_list, seed_to_fertilizer, seed_to_water)

    if info == 4:
        input_list = sentence.split()
        seed_to_light = mapping_info(input_list, seed_to_water, seed_to_light)

    if info == 5:
        input_list = sentence.split()
        seed_to_temperature = mapping_info(input_list, seed_to_light, seed_to_temperature)

    if info == 6:
        input_list = sentence.split()
        seed_to_humidity = mapping_info(input_list, seed_to_temperature, seed_to_humidity)

    if info == 7:
        input_list = sentence.split()
        seed_to_location = mapping_info(input_list, seed_to_humidity, seed_to_location)

print(min(seed_to_location.values()))

In [ ]:
input_text = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

input_text=response.text

In [ ]:
from tqdm import tqdm 
mapping_rules = []
temp = []

temp = [int(i) for i in input_text.split("\n")[0].split(": ")[1].split()]
input_ranges = []
for i in range(len(temp)//2):
    input_ranges.append([temp[2*i], temp[2*i] + temp[2*i+1]-1])
input_ranges = sorted(input_ranges, key=lambda x: x[0])

for sentence in tqdm(input_text.split("\n")):
    input_list = sentence.split()
    if len(input_list) == 3:
        input_list = [int(i) for i in input_list]
        temp.append(input_list)
    elif len(input_list) == 0:
        temp = sorted(temp, key=lambda x: x[1])
        mapping_rules.append(temp.copy())
    else:
        temp = []

mapping_rules.pop(0)
for mapping in mapping_rules:
    convert = []
    for start, end in input_ranges:
        for dest, src, range_length in mapping:
            if start >= src + range_length:
                continue
            elif start < src:
                # Append what is not converted.
                convert.append([start, src - 1])
                start = src
            if end >= src + range_length:
                convert.append([start - src + dest, dest + range_length-1])
                start = src + range_length
            else:
                # Everything fit now
                convert.append([start - src + dest, end - src + dest])
                start = end + 1
                break
        if start <= end:
            convert.append([start, end])
    convert = sorted(convert, key=lambda x: x[0])
    input_ranges = []
    for start, end in convert:
        if input_ranges and start == input_ranges[-1][1] + 1:
            input_ranges[-1] = [input_ranges[-1][0], end]
        else:
            input_ranges.append([start, end])

print(input_ranges[0][0])